The Nobel Prize has been among the most prestigious international awards since 1901. Each year, awards are bestowed in chemistry, literature, physics, physiology or medicine, economics, and peace. In addition to the honor, prestige, and substantial prize money, the recipient also gets a gold medal with an image of Alfred Nobel (1833 - 1896), who established the prize.

![](Nobel_Prize.png)

The Nobel Foundation has made a dataset available of all prize winners from the outset of the awards from 1901 to 2023. The dataset used in this project is from the Nobel Prize API and is available in the `nobel.csv` file in the `data` folder.

In this project, you'll get a chance to explore and answer several questions related to this prizewinning data. And we encourage you then to explore further questions that you're interested in!

In [1]:
# Loading in required libraries
import pandas as pd
import seaborn as sns
import numpy as np

# Start coding here!

In [2]:
nobel = pd.read_csv('data/nobel.csv')

nobel.head()

,year,category,prize,motivation,prize_share,laureate_id,laureate_type,full_name,birth_date,birth_city,birth_country,sex,organization_name,organization_city,organization_country,death_date,death_city,death_country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France


In [3]:
nobel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  1000 non-null   int64 
 1   category              1000 non-null   object
 2   prize                 1000 non-null   object
 3   motivation            912 non-null    object
 4   prize_share           1000 non-null   object
 5   laureate_id           1000 non-null   int64 
 6   laureate_type         1000 non-null   object
 7   full_name             1000 non-null   object
 8   birth_date            968 non-null    object
 9   birth_city            964 non-null    object
 10  birth_country         969 non-null    object
 11  sex                   970 non-null    object
 12  organization_name     736 non-null    object
 13  organization_city     735 non-null    object
 14  organization_country  735 non-null    object
 15  death_date            596 non-null    o

Almost every column is of type object (i.e. string), so we'll need to take that into account when working with any dates or anything in the prize_share column (which could be seen as a number).

## Question 1

What is the most commonly awarded gender and birth country? Storing the string answers as top_gender and top_country.

In [4]:
nobel['sex'].value_counts()

Male      905
Female     65
Name: sex, dtype: int64

In [5]:
top_gender = nobel['sex'].value_counts().index[0]

top_gender

'Male'

In [6]:
nobel['birth_country'].value_counts()

United States of America                        291
United Kingdom                                   91
Germany                                          67
France                                           58
Sweden                                           30
                                               ... 
Austria-Hungary (Ukraine)                         1
Ottoman Empire (Turkey)                           1
Russian Empire (Azerbaijan)                       1
Bosnia (Bosnia and Herzegovina)                   1
French protectorate of Tunisia (now Tunisia)      1
Name: birth_country, Length: 129, dtype: int64

In [7]:
top_country = nobel['birth_country'].value_counts().index[0]

top_country

'United States of America'

## Question 2

What decade had the highest proportion of US-born winners? Store this as an integer called max_decade_usa.

We need to create a column for whether someone is american or not, which will allow us to find the proportion of americans for each decade

In [8]:
nobel['american'] = nobel['birth_country'] == 'United States of America'

nobel['american'].describe()

count      1000
unique        2
top       False
freq        709
Name: american, dtype: object

We also need a column which converts integer year to decade, which is easy to do using floor division

In [9]:
nobel['decade'] = (nobel.year//10)*10

nobel['decade'].unique()

array([1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000,
       2010, 2020])

To get the proportion of American's per decade, we can just do a groupby over the decade column, looking at the mean of the american's variable for each decade, which just consists of booleans, so the mean gives us the proportion of true values

In [10]:
american_decades = nobel.groupby('decade')['american'].mean()

american_decades.round(2)

decade
1900    0.02
1910    0.08
1920    0.07
1930    0.25
1940    0.30
1950    0.29
1960    0.27
1970    0.32
1980    0.32
1990    0.40
2000    0.42
2010    0.31
2020    0.36
Name: american, dtype: float64

That's the 2000's

In [11]:
max_decade_usa = american_decades.sort_values(ascending=False).index[0]

max_decade_usa

2000

## Question 3

What decade and category pair had the highest proportion of female laureates? Store this as a dictionary called max_female_dict where the decade is the key and the category is the value.

Just as we did for americans, we'll need a column to flag that a winner is female

In [12]:
nobel['female'] = nobel['sex'] == 'Female'

nobel['female'].describe()

count      1000
unique        2
top       False
freq        935
Name: female, dtype: object

We already have the decade column due to the work we did for Question 2. We should perhaps inspect the category column, to make sure nothing is wrong with it, then perform the groupby.

In [13]:
nobel['category'].value_counts()

Medicine      227
Physics       225
Chemistry     194
Peace         141
Literature    120
Economics      93
Name: category, dtype: int64

In [14]:
nobel['category'].isna().sum()

0

In [15]:
female_decade_cats = nobel.groupby(['decade', 'category'])['female'].mean()

female_decade_cats.sort_values(ascending=False)

decade  category  
2020    Literature    0.500000
2010    Peace         0.357143
1990    Literature    0.300000
2000    Literature    0.300000
2020    Chemistry     0.300000
                        ...   
1960    Peace         0.000000
1970    Economics     0.000000
        Literature    0.000000
        Physics       0.000000
        Chemistry     0.000000
Name: female, Length: 72, dtype: float64

The answer for most females in a category per decade is Literature in the 2020's (which is impressive since they have .5 and it's still going, so could conceivably be higher at the end)

In [16]:
max_female_index = female_decade_cats.sort_values(ascending=False).index[0]

max_female_index

(2020, 'Literature')

In [17]:
max_female_dict = {max_female_index[0]: max_female_index[1]}

max_female_dict

{2020: 'Literature'}

## Question 4

Who was the first woman to receive a Nobel Prize, and in what category? Save your string answers as first_woman_name and first_woman_category.

We only need cases where 'female' == True, and then we just need to find the minimum year.

In [18]:
female_winners = nobel[nobel['female']]

female_winners.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 19 to 999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  65 non-null     int64 
 1   category              65 non-null     object
 2   prize                 65 non-null     object
 3   motivation            58 non-null     object
 4   prize_share           65 non-null     object
 5   laureate_id           65 non-null     int64 
 6   laureate_type         65 non-null     object
 7   full_name             65 non-null     object
 8   birth_date            65 non-null     object
 9   birth_city            65 non-null     object
 10  birth_country         65 non-null     object
 11  sex                   65 non-null     object
 12  organization_name     28 non-null     object
 13  organization_city     28 non-null     object
 14  organization_country  28 non-null     object
 15  death_date            27 non-null     ob

In [19]:
earliest_female = female_winners[female_winners['year'] == female_winners['year'].min()]

earliest_female

,year,category,prize,motivation,prize_share,laureate_id,laureate_type,full_name,birth_date,birth_city,birth_country,sex,organization_name,organization_city,organization_country,death_date,death_city,death_country,american,decade,female
19,1903,Physics,The Nobel Prize in Physics 1903,"""in recognition of the extraordinary services ...",1/4,6,Individual,"Marie Curie, née Sklodowska",1867-11-07,Warsaw,Russian Empire (Poland),Female,NaN,NaN,NaN,1934-07-04,Sallanches,France,False,1900,True


It's Marie Curie in physics

In [20]:
first_woman_name = earliest_female.full_name.values[0]
first_woman_category = earliest_female.category.values[0]

first_woman_name, first_woman_category

('Marie Curie, née Sklodowska', 'Physics')

## Question 5

Which individuals or organizations have won multiple Nobel Prizes throughout the years? Store the full names in a list named repeat_list.

In [21]:
win_counts = nobel['full_name'].value_counts()

repeat_list = list(win_counts[win_counts > 1].index)

repeat_list

['Comité international de la Croix Rouge (International Committee of the Red Cross)',
 'Linus Carl Pauling',
 'John Bardeen',
 'Frederick Sanger',
 'Marie Curie, née Sklodowska',
 'Office of the United Nations High Commissioner for Refugees (UNHCR)']